We will analyze https://research.google.com/audioset///dataset/emergency_vehicle.html

# Preample

In [3]:
import json
import pandas as pd
import numpy as np
import youtube_dl
import os
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from multiprocessing import Process
import multiprocessing

# Load data

In [4]:
# paths

path_ontology = "/nfs/students/summer-term-2020/project-4/data/dataset1/ontology/ontology.json"
path_train_unbalanced = "/nfs/students/summer-term-2020/project-4/data/dataset1/audioSetCsv/unbalanced_train_segments.csv"
path_train_balanced = "/nfs/students/summer-term-2020/project-4/data/dataset1/audioSetCsv/balanced_train_segments.csv"
path_eval_balanced = "/nfs/students/summer-term-2020/project-4/data/dataset1/audioSetCsv/eval_segments.csv"

In [5]:
def loadJsonFile(path):
    if path[-5:] == '.json':
        return json.load(open(path))
    
def loadCsvFile(path):
    if path[-4:] == '.csv':
        names = ['YTID', 'start_seconds', 'end_seconds', 'positive_labels']
        return pd.read_csv(open(path), sep=', ', header=None, index_col=0, 
                           skiprows=3, names=names, engine='python')

In [6]:
ontoloy = loadJsonFile(path_ontology)
train_unbalanced = loadCsvFile(path_train_unbalanced)
train_balanced = loadCsvFile(path_train_balanced)
eval_balanced = loadCsvFile(path_eval_balanced)

# Analyzing ontology

## Find entry

In [13]:
for entry in ontoloy:
    if 'Human ' in entry['name']:
        print("\t " + entry['name'] + " (" + entry['id'] + ")")

	 Human sounds (/m/0dgw9r)
	 Human voice (/m/09l8g)
	 Human locomotion (/m/0bpl036)
	 Human group actions (/t/dd00012)


## Analyze ontology towards Emergency Vehicle dataset

In [18]:
def findChildren(ontology_id):
    result = []
    for entry in ontoloy:
        if entry['id'] == ontology_id:
            result.append(ontology_id)
            for child_id in entry['child_ids']:
                result.extend(findChildren(child_id))
    return result

In [19]:
supercategories = [
    '/m/04rlf', # Music: https://research.google.com/audioset/ontology/music_1.html
    '/m/0jbk', # Animals: https://research.google.com/audioset/ontology/animal_1.html
    '/m/059j3w', # Natural sounds: https://research.google.com/audioset/ontology/natural_sounds_1.html
    '/m/09l8g', # Human voice: https://research.google.com/audioset/ontology/human_voice_1.html
    # negative classes:
    '/m/07pp_mv', # Alarms: https://research.google.com/audioset///ontology/alarm_1.html
    '/m/012f08' # https://research.google.com/audioset////ontology/motor_vehicle_road_1.html
]

In [28]:
positive_ids = []
interpolation_ids = {}

# https://research.google.com/audioset///ontology/emergency_vehicle_1.html
positive_ids.extend(findChildren('/m/03j1ly')) 

for interpolation_id in supercategories:
    interpolation_ids[interpolation_id]= ([x for x in findChildren(interpolation_id) if x not in positive_ids])

In [30]:
def printChildren(ontology_id, depth, list_of_ids):
    for entry in ontoloy:
        if entry['id'] == ontology_id and ontology_id in list_of_ids: 
            print(''.join(["\t" for _ in range(depth)]) + entry['name'] + " (" + entry['id'] + ")")
            for child_id in entry['child_ids']:
                printChildren(child_id, depth+1, list_of_ids)

In [32]:
print("Positive classes:")
# https://research.google.com/audioset///ontology/emergency_vehicle_1.html
printChildren('/m/03j1ly', 1, positive_ids)
    
print()
print("Interpolation classes:")
for interpolation_id in supercategories:
    printChildren(interpolation_id, 1, interpolation_ids[interpolation_id])

Positive classes:
	Emergency vehicle (/m/03j1ly)
		Police car (siren) (/m/04qvtq)
		Ambulance (siren) (/m/012n7d)
		Fire engine, fire truck (siren) (/m/012ndj)

Interpolation classes:
	Music (/m/04rlf)
		Musical instrument (/m/04szw)
			Plucked string instrument (/m/0fx80y)
				Guitar (/m/0342h)
					Electric guitar (/m/02sgy)
					Bass guitar (/m/018vs)
					Acoustic guitar (/m/042v_gx)
					Steel guitar, slide guitar (/m/06w87)
					Tapping (guitar technique) (/m/01glhc)
					Strum (/m/07s0s5r)
				Banjo (/m/018j2)
				Sitar (/m/0jtg0)
				Mandolin (/m/04rzd)
				Zither (/m/01bns_)
				Ukulele (/m/07xzm)
			Keyboard (musical) (/m/05148p4)
				Piano (/m/05r5c)
					Electric piano (/m/01s0ps)
						Clavinet (/m/025cbm)
						Rhodes piano (/m/0bxl5)
				Organ (/m/013y1f)
					Electronic organ (/m/03xq_f)
					Hammond organ (/m/03gvt)
				Synthesizer (/m/0l14qv)
					Sampler (/m/01v1d8)
					Mellotron (/m/0gkd1)
				Harpsichord (/m/03q5t)
			Percussion (/m/0l14md)
				Drum kit (/m/02hnl)

# Creating dataset

In [37]:
def binaryClassification(labels, positive_ids, negative_ids):
    containsPositiveLabel = False
    containsNegativeLabel = False
    
    for label in labels:
        if label in positive_ids:
            containsPositiveLabel = True
    
        if label in negative_ids:
            containsNegativeLabel = True
            
    return containsPositiveLabel, containsNegativeLabel

In [38]:
def isPositiveSample(labels, positive_ids, negative_ids):
    containsPositiveLabel, containsNegativeLabel = binaryClassification(labels, positive_ids, negative_ids)
    # don't exclude samples with negative labels, 
    # as EM sounds sometimes labeled as siren as well (we deal with multi-labeling here)
    return containsPositiveLabel

In [39]:
def isNegativeSample(labels, positive_ids, negative_ids):
    containsPositiveLabel, containsNegativeLabel = binaryClassification(labels, positive_ids, negative_ids)
    return not containsPositiveLabel and containsNegativeLabel

In [40]:
def getBinaryDatasetSamples(dataset, positive_ids, negative_ids):
    positives = pd.DataFrame(data=None, columns=dataset.columns)
    negatives = pd.DataFrame(data=None, columns=dataset.columns)

    for index, row in dataset.iterrows():
        labels = row['positive_labels'].split(',')

        isPositive = isPositiveSample(labels, positive_ids, negative_ids)
        isNegative = isNegativeSample(labels, positive_ids, negative_ids)

        if isPositive and not isNegative:
            positives.loc[index]=row
        elif not isPositive and isNegative:
            negatives.loc[index]=row
            
    return positives, negatives

In [57]:
interpolation_categories_dataset = {}

for interpolation_id in interpolation_ids:
    negative_ids = interpolation_ids[interpolation_id]

    #_, train_unbalanced_negatives = getBinaryDatasetSamples(train_unbalanced, positive_ids, negative_ids)
    _, train_balanced_negatives = getBinaryDatasetSamples(train_balanced, positive_ids, negative_ids)
    #_, eval_balanced_negatives = getBinaryDatasetSamples(eval_balanced, positive_ids, negative_ids)
    
    #interpolation_categories_dataset.setdefault(interpolation_id, []).append(train_unbalanced_negatives)
    interpolation_categories_dataset.setdefault(interpolation_id, []).append(train_balanced_negatives)
    #interpolation_categories_dataset.setdefault(interpolation_id, []).append(eval_balanced_negatives)
    print("Done: " + interpolation_id)

Done: /m/04rlf
Done: /m/0jbk
Done: /m/059j3w
Done: /m/09l8g
Done: /m/07pp_mv
Done: /m/012f08


In [68]:
category_size = 200 # we will remove those that we cannot download later on

final_interpolation_dataset = None

for interpolation_id in interpolation_categories_dataset:
    interpolation_categories_dataset[interpolation_id][0]['super_category'] = interpolation_id

final_interpolation_dataset = pd.concat([df[0].sample(category_size, random_state=42) for df in interpolation_categories_dataset.values()])

# Downloading

In [71]:
ydl_opts = {
    'quiet':'True',
    'format': 'bestaudio/best',
    'ignoreerrors':'True',
    'no_warnings':'True',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
    'outtmpl':'tmp/%(id)s.%(ext)s',
    }

In [72]:
def downloadDataset(dataset, save_path):
    for index, row in dataset.iterrows():
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download(['http://www.youtube.com/watch?v=' + index])

        name = index + '.wav'
        path_downloaded = './tmp/' + name
        path_final = save_path + name
        start = row[0]
        end = row[1]
        
        if os.path.exists(path_downloaded):
            ffmpeg_extract_subclip(path_downloaded, start, end,targetname=path_final)
            os.remove(path_downloaded)

In [73]:
def downloadDatasetParallelized(dataset, save_path):
    num_cpus = multiprocessing.cpu_count()
    listOfDfs = [dataset.loc[idx] for idx in np.array_split(dataset.index,num_cpus)]
    print("Mean chunk size: " + str(np.mean([len(x) for x in listOfDfs])))
    
    processes = []
    for chunk in listOfDfs:
        p = Process(target=downloadDataset, args=(chunk,save_path))
        processes.append(p)
        p.start()
        
    for p in processes:
        p.join()
        
    dataset.to_csv(save_path + "meta.csv")

In [76]:
path = "/nfs/students/summer-term-2020/project-4/data/interpolation_data/download/"

In [77]:
downloadDatasetParallelized(final_interpolation_dataset, path)

Mean chunk size: 66.2


ERROR: 1_iz3BvTJug: YouTube said: This video is unavailable.
Sorry about that.
ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Running:
>>> "+ " ".join(cmd)Moviepy - Running:
>>> "+ " ".join(cmd)Moviepy - Running:
>>> "+ " ".join(cmd)




ERROR: o08cGjueDXM: YouTube said: This video has been removed for violating YouTube's policy on violent or graphic content.
Sorry about that.


Moviepy - Command successful
Moviepy - Command successful
Moviepy - Command successful
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Command successful
Moviepy - Command successful


...

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


ERROR: 4HeUmlSg-jY: YouTube said: This video is unavailable.
Sorry about that.


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
